In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_beam_projections, Machine

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors.multiplication import ChargeWeighter
from PyHEADTAIL.feedback.processors.linear_transform import Averager
from PyHEADTAIL.feedback.processors.misc import Bypass
from PyHEADTAIL.feedback.processors.register import Register, TurnDelay, UncorrectedDelay

np.random.seed(0)

# 006 Separate pickup and kicker with multi bunch beam

In this test/example, a pickup and a kicker are located separately in the one turn map and beam consists of multiple bunches. The test is identical to the test *002\_single\_bunch\_\-\_separate\_pickup\_and\_kicker.ipynb* expect that multibunch PyHEADTAIL beam is used and one_turn_map objects are run in the MPI mode.

## Basic parameters and elements for the simulations

In [ ]:
%%capture

n_macroparticles = 1000
n_slices = 20
n_segments = 5
n_sigma_z = 3
# n_sigma_z = 6

n_turns = 150

machine = Machine(n_segments= n_segments)

first_index = 10 #[buckets]
batch_spacing = 100  #[buckets]
n_batches = 2
n_bunches_per_batch = 20
bunch_spacing = 10 #[buckets]

batch_separation = batch_spacing+n_bunches_per_batch* bunch_spacing

filling_scheme = []
for j in xrange(n_batches):
    for i in xrange(n_bunches_per_batch):
        filling_scheme.append(first_index + i * bunch_spacing + j*batch_separation)


print filling_scheme

beam_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles, n_slices,n_sigma_z,
                                                             filling_scheme=filling_scheme);


In [ ]:
print('Number of bunches: ' + str(len(beam_ref.split())))

## Initial bunch kicks
Creates an artificially (bunch-by-bunch) kicked beam, which will be damped by using different feedback models. 

In [ ]:
bunch_list = beam_ref.split()

n_bunches = len(bunch_list)

kick_x = 0.003*(-1.0+2*np.random.rand(n_bunches))
kick_y = 0.003*(-1.0+2*np.random.rand(n_bunches))

for i in xrange(n_bunches):
    bunch_list[i].x = bunch_list[i].x + kick_x[i]
    bunch_list[i].y = bunch_list[i].y + kick_y[i]

beam_ref = sum(bunch_list)

## Feedback settings

In [ ]:
feedback_gain = 0.1
# feedback_gain = (0.1,0.4)

# delay (a number of turns) before the pickup signal is used to the correction kick calculations.
delay = 1

# a number of values used to calculate the correct signal
n_values = 2

## Reference data
Tracks a bunch by using an ideal bunch-by-bunch feedback system presented in the previous test (001_ideal_feedbacks.ipynb). This data are used as a reference data for the feedback models including delay and separate pickup(s) and kicker(s)

In [ ]:
beam_ref_data = copy.deepcopy(beam_ref)
tracker_ref_data = BunchTracker(beam_ref_data)
slicer_ref_data = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_ref_data,processors_bunch_x,processors_bunch_y, mpi=True)
one_turn_map = [i for i in trans_map] + [feedback_map] #  + [long_map]

track(n_turns, beam_ref_data,one_turn_map ,tracker_ref_data)

## Case 1: turn delay

In [ ]:
beam_delay_algorithm = copy.deepcopy(beam_ref)
tracker_delay_algorithm = BunchTracker(beam_delay_algorithm)
slicer_delay_algorithm = copy.deepcopy(slicer_ref)



processors_delay_algorithm_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    TurnDelay(delay, machine.Q_x, n_values,additional_phase_advance=0.)
]
processors_delay_algorithm_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    TurnDelay(delay, machine.Q_y, n_values,additional_phase_advance=0.)
]

feedback_map = OneboxFeedback(feedback_gain,slicer_delay_algorithm,
                              processors_delay_algorithm_x,processors_delay_algorithm_y, mpi=True)
one_turn_map = [feedback_map] + [i for i in trans_map] # + [long_map]

track(n_turns, beam_delay_algorithm,one_turn_map,tracker_delay_algorithm)


## Case 2: separate pickup and kicker 

In [ ]:
# Beta function values in the pickup and the kicker locations
pickup_beta_x = machine.beta_x
pickup_beta_y = machine.beta_y

kicker_beta_x = machine.beta_x
kicker_beta_y = machine.beta_y

In [ ]:
pickup_location_x = 1.*2.*pi/float(n_segments)*machine.Q_x
pickup_location_y = 1.*2.*pi/float(n_segments)*machine.Q_y

kicker_location_x = 2.*2.*pi/float(n_segments)*machine.Q_x
kicker_location_y = 2.*2.*pi/float(n_segments)*machine.Q_y

In [ ]:
beam_separate_algorithm = copy.deepcopy(beam_ref)
tracker_separate_algorithm = BunchTracker(beam_separate_algorithm)
slicer_separate_algorithm = copy.deepcopy(slicer_ref)

# Exactly same signal processors as in the previous example are used in the pickup
processors_pickup_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    Register(n_values, machine.Q_x, delay)
]
processors_pickup_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    Register(n_values, machine.Q_y, delay)
]

pickup_map = PickUp(slicer_separate_algorithm,processors_pickup_x,processors_pickup_y, pickup_location_x,
                    pickup_beta_x, pickup_location_y, pickup_beta_y, mpi=True)


# In this example, the signals are not modified in the kicker, i.e. only bypass processors are used
processors_kicker_x = [Bypass()]
processors_kicker_y = [Bypass()]

# A list of references to registers from the signal processors(s) of the pickup(s)
registers_x = [processors_pickup_x[-1]]
registers_y = [processors_pickup_y[-1]]

kicker_map = Kicker(feedback_gain, slicer_separate_algorithm, processors_kicker_x, processors_kicker_y,
                    registers_x, registers_y, kicker_location_x, kicker_beta_x, kicker_location_y, kicker_beta_y,
                    mpi=True)

one_turn_map = [trans_map[0]] + [pickup_map] + [trans_map[1]] + [kicker_map]
for element in trans_map[2:]:
    one_turn_map += [element]
# tone_turn_map += [long_map]
    
track(n_turns, beam_separate_algorithm,one_turn_map,tracker_separate_algorithm)

## Results comparison

In [ ]:
compare_traces([tracker_ref_data,tracker_delay_algorithm, tracker_separate_algorithm],
               ['Reference', 'Delayed, algorithm', 'Separate, algorithm'])
compare_beam_projections([beam_ref_data,beam_delay_algorithm, beam_separate_algorithm],
               ['Reference', 'Delayed, algorithm', 'Separate, algorithm'])

Jani Komppula, CERN, 2017